# Exploring Twitter API

## Importing Packages

In [15]:
import pandas as pd
import json
import re
import requests
import pickle
import twint

%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../py")
from utils import *
from config import keys

## Aristotle University Dataset

Citation:

Founta, A., Djouvas, C., Chatzakou, D., Leontiadis, I., Blackburn, J., Stringhini, G., Vakali, A., Sirivianos, M. and Kourtellis, N., 2018. Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior \[Data file\]. ArXiv. Retrieved from: https://dataverse.mpi-sws.org/dataset.xhtml?persistentId=doi:10.5072/FK2/ZDTEMN

Publication: https://arxiv.org/pdf/1802.00393.pdf

In [2]:
df = pd.read_csv('../data/hatespeechtwitter.csv')
df.head()

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   80000 non-null  int64 
 1   maj_label  79996 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [4]:
df.columns = ['id', 'label']
df.label.value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: label, dtype: int64

In [5]:
df_hateful = df[df['label']=='hateful']
df_hateful

,id,label
5,850433664890544128,hateful
11,850449456445235200,hateful
12,850100742035836929,hateful
13,847945888995708928,hateful
21,847804507367100416,hateful
...,...,...
79853,849246676854484992,hateful
79909,849635522389651456,hateful
79917,850456293181538304,hateful
79922,850308804663181312,hateful


In [6]:
hate_ids = group_list(list(df_hateful.id))
len(hate_ids)

37

In [7]:
hate_ids[0]

'850433664890544128,850449456445235200,850100742035836929,847945888995708928,847804507367100416,850090319165050880,848600351381098496,850672203372916736,849303652280020995,849300057782456321,848686686930382848,850380636300820480,847652946217009155,850562987870105600,849763234743808002,848835698006401024,849310820345532416,850215875680522240,847860379682320384,849922895132459008,847917841659473920,849042540091387904,847663964657860609,848110494422630400,847520418793762816,849623568619057152,847591914874572800,848705682937450496,848594131257577473,849751289399566336,847811708995481600,850741375817601025,848997472273936384,847457248355811328,847947877058052096,850074930263859201,848080463214374912,849965110806409216,848653006707011584,850179691386249217,849960731965620224,847693215717249024,848877670389755904,848721914898124808,850730957145616385,849828636563841029,847566791014363136,850008916129964033,848759395215265792,847909058803892224,847498662905307136,849700412475064320,85044523278

In [14]:
url = "https://api.twitter.com/2/tweets?ids=847661947159891972,847799130277675008,848933211375779840&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username"
payload={}
headers = {'Authorization': 'Bearer ' + keys['bearer_token'], 'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
r = requests.request("GET", url, headers=headers, data=payload)
data = r.json()

In [16]:
df_hate = tweets_request(hate_ids)

100%|██████████| 37/37 [00:22<00:00,  1.66it/s]


In [17]:
df_hate = df_hate.reset_index(drop=True)
df_hate

,created_at,text,id,public_metrics,entities,geo
0,2017-04-07T19:42:40.000Z,I hate er chase because if the Bitch that work...,850433664890544128,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN
1,2017-04-06T21:39:45.000Z,RT @nyctophil3: Pineapples do not belong on pi...,850100742035836929,"{'retweet_count': 5, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",NaN
2,2017-03-31T13:35:20.000Z,Niggas keep talking about women wearing weave ...,847804507367100416,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN
3,2017-04-06T20:58:20.000Z,@vappywave idiot that's not gonna work. you go...,850090319165050880,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN
4,2017-04-03T00:00:48.000Z,RT @ayevonnn: bruh i fucking hate people like ...,848686686930382848,"{'retweet_count': 5, 'reply_count': 0, 'like_c...","{'urls': [{'start': 53, 'end': 76, 'url': 'htt...",NaN
...,...,...,...,...,...,...
1739,2017-04-08T13:25:50.000Z,Baekhyun won 'Most Popular Artist of Korea'\n\...,850701219568001025,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 0, 'end': 7, 'proba...",NaN
1740,2017-04-04T17:42:11.000Z,#TripleTalaqArrest 2fgt dis social menace #Tri...,849316180670259205,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'hashtags': [{'start': 0, 'end': 18, 'tag': '...",NaN
1741,2017-04-05T17:26:51.000Z,White nationalists' latest tactic to recruit c...,849674709763358720,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'urls': [{'start': 85, 'end': 108, 'url': 'ht...",NaN
1742,2017-04-04T13:06:00.000Z,We are in a comfortable lead fight against gal...,849246676854484992,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 53, 'end': 66, 'tag': ...",NaN


In [18]:
pickle.dump(df_hate, open("../pickle/aristotle_2.pickle", "wb"))


# University of Copenhagen Dataset

** Reference: **

Waseem, Z., Hovy, D. (2016). Hateful Symbols or Hateful People? Predictive Features for Hate Speech Detection on Twitter \[Data file\]. In: *Proceedings of the NAACL Student Research Workshop*. San Diego, Calfornia: Association for Computational Linguistics, pages 88-93. Retrieved from: https://github.com/ZeerakW/hatespeech.

Publication:  https://www.aclweb.org/anthology/N16-2013.pdf

In [46]:
df2 = pd.read_csv('../data/NAACL_SRW_2016.csv')
df2.columns = ['id', 'label']
df2.head()

,id,label
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [47]:
df2.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [21]:
df_racsex = df2[(df2['label']=='racism') | (df2['label']=='sexism')]

In [22]:
racsex_id = group_list(list(df_racsex.id))
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:25<00:00,  2.08it/s]


In [23]:
df_rac_sex = df_rac_sex.reset_index(drop=True)
df_rac_sex

,text,id,geo,created_at,public_metrics,entities
0,Drasko they didn't cook half a bird you idiot ...,572341498827522049,"{'place_id': '017453ae077eafd3', 'coordinates'...",2015-03-02T10:23:41.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 46, 'end': 50, 'tag': ..."
1,Hopefully someone cooks Drasko in the next ep ...,572340476503724032,NaN,2015-03-02T10:19:37.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 24, 'end': 29, 'pro..."
2,of course you were born in serbia...you're as ...,572334712804384768,NaN,2015-03-02T09:56:43.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 27, 'end': 32, 'pro..."
3,These girls are the equivalent of the irritati...,572332655397629952,NaN,2015-03-02T09:48:33.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 95, 'end': 99, 'tag': ..."
4,RT @YesYoureRacist: At least you're only a tin...,446460991396917248,NaN,2014-03-20T01:39:29.000Z,"{'retweet_count': 46, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 18, 'usernam..."
...,...,...,...,...,...,...
2721,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ...",569990621315919872,NaN,2015-02-23T22:42:08.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","{'urls': [{'start': 94, 'end': 116, 'url': 'ht..."
2722,RT @DufresneFrank: “@gabystama: @DufresneFrank...,570244879265206272,NaN,2015-02-24T15:32:28.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 17, 'usernam..."
2723,RT @BoycottBrandy: @ActionFlickDoc this person...,571013698111860736,NaN,2015-02-26T18:27:29.000Z,"{'retweet_count': 3, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 17, 'usernam..."
2724,RT @Lyall: Patriarchy™ will expel me if I divu...,571304517930774528,NaN,2015-02-27T13:43:06.000Z,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 9, 'username..."


In [24]:
pickle.dump(df_rac_sex, open("../pickle/copenhagen_2.pickle", "wb"))

# Georgia Tech Dataset

In [27]:
df3 = pd.read_csv("../data/hate.csv")
df3.columns = ['id', 'user', 'hate', 'chate', 'neutral', 'label']

In [28]:
df_anti_asian = df3.sort_values(by = ['hate'], ascending=False)
df_anti_asian.label = 1
df_anti_asian

,id,user,hate,chate,neutral,label
560314,1240976182494908416,884055068265164801,0.999974,0.000520,0.000382,1
515476,1238239331879464960,1136405401770106880,0.999966,0.000625,0.000105,1
396712,1247813175996792834,1234197517006450688,0.999959,0.003865,0.000018,1
424027,1245317927823233024,1232772980763054081,0.999958,0.001476,0.000318,1
380359,1241373862756888577,907414928,0.999952,0.018423,0.000788,1
...,...,...,...,...,...,...
596002,1236335083738537985,4384833496,0.500002,0.004678,0.297274,1
334098,1250769238274572291,1199735925355556864,0.500002,0.087211,0.045595,1
889914,1232245722831052801,1053001766806200322,0.500001,0.004162,0.253713,1
198091,1249377836646924291,223299902,0.500001,0.003931,0.210026,1


In [29]:
anti_asian_ids = group_list(list(df_anti_asian.id))
len(anti_asian_ids)

8913

In [30]:
df_anti_asian = df3.sort_values(by = ['hate'], ascending=False)
df_anti_asian.label = 1
df_anti_asianasian_100 = anti_asian_ids[0:100]
asian_200 = anti_asian_ids[100:200]
asian_300 = anti_asian_ids[200:300]

In [31]:
df_1 = tweets_request(asian_100)

100%|██████████| 100/100 [01:08<00:00,  1.46it/s]


In [32]:
df_2 = tweets_request(asian_200)

100%|██████████| 100/100 [01:04<00:00,  1.54it/s]


In [33]:
df_3 = tweets_request(asian_300)

100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


In [34]:
df_asian = pd.concat([df_1, df_2, df_3], axis=0)
df_asian = df_asian.reset_index(drop=True)
df_asian

,public_metrics,text,entities,created_at,id,geo
0,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Not the people of China but the Chinese Commun...,"{'hashtags': [{'start': 56, 'end': 65, 'tag': ...",2020-04-04T15:40:23.000Z,1246462627158491136,NaN
1,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",182 countries are currently suffering the scou...,"{'hashtags': [{'start': 82, 'end': 88, 'tag': ...",2020-04-05T23:59:06.000Z,1246950520914313216,NaN
2,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",OH SHIT OH SHIT OH SHIT OH SHIT OH SHIT OH SHI...,"{'urls': [{'start': 275, 'end': 298, 'url': 'h...",2020-03-17T18:10:23.000Z,1239977396473860096,{'place_id': '006b2a835a30b702'}
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@RealJamesWoods Either #WuhanVirus or #Chinese...,"{'hashtags': [{'start': 23, 'end': 34, 'tag': ...",2020-03-18T18:50:54.000Z,1240349980738506752,NaN
4,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@NYGovCuomo #China lied about the #WuhanVirus ...,"{'hashtags': [{'start': 12, 'end': 18, 'tag': ...",2020-04-05T11:56:23.000Z,1246768643913814019,NaN
...,...,...,...,...,...,...
16764,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@Oceanle15689731 corona is from China because ...,"{'annotations': [{'start': 32, 'end': 36, 'pro...",2020-04-12T15:41:47.000Z,1249362085248524289,NaN
16765,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",President Donald Trump and Congress should tel...,"{'annotations': [{'start': 10, 'end': 21, 'pro...",2020-03-18T21:30:38.000Z,1240390179048824833,NaN
16766,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",@almostzdq @cgtnamerica @CGTNOfficial Hard to ...,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",2020-04-04T00:56:49.000Z,1246240273455296512,NaN
16767,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@MarshaBlackburn @RepJimBanks @teammoulton #Ch...,"{'annotations': [{'start': 88, 'end': 92, 'pro...",2020-03-25T13:46:10.000Z,1242810006715527168,NaN


In [28]:
pickle.dump(df_asian, open("../pickle/georgia.pickle", "wb"))

# HASOC 2019 and 2020 Dataset

In [35]:
hasoc = pd.read_csv("../data/hasoc2019_en_test-2919.tsv", delimiter="\t")
hasoc

,text_id,text,task_1,task_2,task_3
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...,NOT,NONE,NONE
1,hasoc_en_416,68.5 million people have been forced to leave ...,NOT,NONE,NONE
2,hasoc_en_207,"You came, you saw .... we will look after the ...",NOT,NONE,NONE
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...,NOT,NONE,NONE
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...,HOF,PRFN,UNT
...,...,...,...,...,...
1148,hasoc_en1_3958,Each of the divorce must be pronounced with a ...,NOT,NONE,NONE
1149,hasoc_en1_4648,He is 62-yr-old Harswaroop Chauhan. His only s...,NOT,NONE,NONE
1150,hasoc_en1_4832,"Peace of graveyard or peace of mind, @asadowai...",NOT,NONE,NONE
1151,hasoc_en1_3721,The humble #tribute on the sacrifice day of #J...,NOT,NONE,NONE


In [37]:
english = pd.read_csv("../data/english_dataset.tsv", delimiter="\t")
english

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
...,...,...,...,...,...
5847,hasoc_en_5848,"@davidfrum @trueblueusa1 That's cute and all, ...",HOF,PRFN,UNT
5848,hasoc_en_5849,a recession issa comin' #maga #magamyass #fuck...,NOT,NONE,NONE
5849,hasoc_en_5850,#DoctorsFightBack Will 'The Mad n Irrational ...,HOF,OFFN,TIN
5850,hasoc_en_5851,#ShiningIndia #educatedindia or more like RUND...,HOF,OFFN,UNT


In [39]:
neutral = english[english['task_1'] == 'NOT']
neutral

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE
...,...,...,...,...,...
5824,hasoc_en_5825,@DOlusegun @elrufai Hope they don't becomes #D...,NOT,NONE,NONE
5825,hasoc_en_5826,@gtconway3d @KellyannePolls @realDonaldTrump H...,NOT,NONE,NONE
5827,hasoc_en_5828,@washingtonpost This is something everyone sho...,NOT,NONE,NONE
5828,hasoc_en_5829,Sorry about that. Has anybody got a #dickhead ...,NOT,NONE,NONE


In [42]:
hate = english[english['task_2'] == 'HATE']
hate

,text_id,text,task_1,task_2,task_3
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
11,hasoc_en_12,I don’t know how much more I can take! 45 is a...,HOF,HATE,TIN
23,hasoc_en_24,#ShameOnICC 1. ICC on Dhoni's gloves ...,HOF,HATE,TIN
27,hasoc_en_28,Disgusting! And they don’t change their cultur...,HOF,HATE,TIN
34,hasoc_en_35,"@TheRealOJ32 Of all the retired NFL players, w...",HOF,HATE,TIN
...,...,...,...,...,...
5840,hasoc_en_5841,Full of fools.. ICC @ICCMediaComms @ICC @crick...,HOF,HATE,TIN
5842,hasoc_en_5843,"The US-Mexican border, where established moral...",HOF,HATE,TIN
5844,hasoc_en_5845,@ICC All Eng matches are on dry pitches &amp; ...,HOF,HATE,TIN
5845,hasoc_en_5846,It seems the Rain &amp; thunder god only like ...,HOF,HATE,TIN


In [45]:
offensive = english[english['task_2'] == 'OFFN']
offensive

,text_id,text,task_1,task_2,task_3
49,hasoc_en_50,@TheRealOJ32 The world will rejoice when you d...,HOF,OFFN,TIN
91,hasoc_en_92,@realDonaldTrump Destroy our country? More li...,HOF,OFFN,TIN
102,hasoc_en_103,So it looks like the lying incompetent homopho...,HOF,OFFN,TIN
129,hasoc_en_130,@UROCKlive1 He’s had the worst numbers of any ...,HOF,OFFN,TIN
144,hasoc_en_145,@realDonaldTrump This is one of the worst time...,HOF,OFFN,TIN
...,...,...,...,...,...
5826,hasoc_en_5827,@briantylercohen @mattgaetz is such a #doucheb...,HOF,OFFN,TIN
5838,hasoc_en_5839,@ImranKhanPTI if u and ur govt have little sha...,HOF,OFFN,TIN
5841,hasoc_en_5842,"@TheRealOJ32 You’re really stabbin...er, grabb...",HOF,OFFN,TIN
5849,hasoc_en_5850,#DoctorsFightBack Will 'The Mad n Irrational ...,HOF,OFFN,TIN


In [ ]:

def tweets_request(tweets_ids):
    """
    Make a requests to Tweeter API
    """
    df_lst = []
    
    for batch in tqdm(tweets_ids):
        url = "https://api.twitter.com/2/tweets?ids={}&tweet.fields=created_at&expansions=author_id&user.fields=created_at".format(batch)
        payload={}
        headers = {'Authorization': 'Bearer ' + config.keys['bearer_token'],
        'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
        r = requests.request("GET", url, headers=headers, data=payload)
        data = r.json()
        if 'data' in data.keys():
            df_lst.append(pd.DataFrame(data['data']))
    
    return pd.concat(df_lst)

In [3]:
from twython import Twython

keys = dict(
    
    app_id = "20913304"
    
    api_key = 'JusiYpq36K4OJBekSt5OWbSRj',

    api_secret = 'z08v32SwpbqfBWPOJOuL1aI5lWzsBRy38BiLsVetK0wNLRmQYh',

    access_token = '1299634175792775168-jpKi66cfOvOUEJQwMrXHWIZskDyNXG', 

    access_token_secret = 'u4ZA1BWHRIs1Gyx575QXO4Q8hmVsRBtmwlOCyB0Rdo1vS',

    bearer_token = 'AAAAAAAAAAAAAAAAAAAAAC%2BENQEAAAAAZSGS82Ee99vbsQYf0V4FvKz%2F5VM%3D4tuNSKj6s2cxthns7IQtzA19DDsFz9Rxj1m6r6PD9rBxB7mies'

)

In [26]:
APP_KEY = 'JusiYpq36K4OJBekSt5OWbSRj'
APP_SECRET = 'z08v32SwpbqfBWPOJOuL1aI5lWzsBRy38BiLsVetK0wNLRmQYh'
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [27]:
print(ACCESS_TOKEN)

AAAAAAAAAAAAAAAAAAAAAC%2BENQEAAAAAZSGS82Ee99vbsQYf0V4FvKz%2F5VM%3D4tuNSKj6s2cxthns7IQtzA19DDsFz9Rxj1m6r6PD9rBxB7mies


In [28]:
twitter = Twython(APP_KEY, access_token = ACCESS_TOKEN)

In [20]:
# twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
APP_KEY = "C8x0cgMxMVo0XWO2x08COIZAP"
APP_SECRET = "Wmi4DPwEYa7c1okNCtIdLeGf9wFkXEvDWULN9Lec28zFwaaEAm"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [21]:
twitter = Twython(APP_KEY, APP_SECRET)
auth = twitter.get_authentication_tokens(callback_url="https://www.examsherpa.com")

In [22]:
OAUTH_TOKEN = auth['oauth_token']
OAUTH_TOKEN_SECRET = auth['oauth_token_secret']


In [23]:
print(OAUTH_TOKEN)

ASKNyAAAAAABPxyYAAABeYCtFKk


In [24]:
print(OAUTH_TOKEN_SECRET)

wiLSW4JSNwjrtyoMs6Wh88H0qxhsoH0Q


In [19]:
oauth_verifier = requests.GET['oauth_verifier']

AttributeError: module 'requests' has no attribute 'GET'

In [25]:
auth['auth_url']

'https://api.twitter.com/oauth/authenticate?oauth_token=ASKNyAAAAAABPxyYAAABeYCtFKk'

In [15]:
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [16]:
twitter.verify_credentials()

TwythonAuthError: Twitter API returned a 401 (Unauthorized), Invalid or expired token.